In [3]:
import torch
import torch.nn as nn
import pydicom
import os
import numpy as np
from PIL import Image

# CT图像序列的预处理
def load_ct_sequence(dicom_folder, target_size=(256, 256)):
    ct_images = []
    # 读取文件夹中的所有.dcm文件
    dicom_paths = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.endswith('.dcm')]
    dicom_paths.sort()  # 按文件名排序，确保读取顺序正确
    
    for path in dicom_paths:
        dicom_img = pydicom.dcmread(path).pixel_array
        dicom_img = Image.fromarray(dicom_img).convert("RGB")
        dicom_img = dicom_img.resize(target_size)  # 调整图像大小为目标尺寸
        ct_images.append(np.array(dicom_img))  # 添加到图像列表
    
    ct_images = np.stack(ct_images, axis=0)  # 将图像堆叠成一个3D数组 (D, H, W, C)
    return ct_images

# 定义处理CT序列的3D卷积网络，输出1x256特征
class CTNet3D(nn.Module):
    def __init__(self):
        super(CTNet3D, self).__init__()
        self.conv1 = nn.Conv3d(3, 32, kernel_size=(3, 3, 3), stride=1, padding=1)  # 3D卷积
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)  # 3D池化层
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=1, padding=1)
        
        # 临时定义了一个全连接层，以后会根据输入数据的实际维度动态调整
        self.fc = None

    def forward(self, ct_images):
        print(f"输入数据的形状: {ct_images.shape}")  # 打印输入数据的形状
        x = self.conv1(ct_images)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        print(f"卷积和池化后的形状: {x.shape}")  # 打印卷积后的形状
        
        # 动态计算Linear层输入维度
        x = x.view(x.size(0), -1)  # 将多维数据拉平为一维
        print(f"拉平后的形状: {x.shape}")  # 打印拉平后的形状
        
        if self.fc is None:
            # 根据卷积输出的维度动态定义全连接层
            self.fc = nn.Linear(x.size(1), 256)  # 动态获取输入维度
            
        x = self.fc(x)  # 输出256维的特征
        return x

# 特征提取并保存为.pt文件
def extract_features_and_save(ct_dicom_folder, save_dir="output_features"):
    # 加载CT图像序列
    ct_images = load_ct_sequence(ct_dicom_folder)
    
    # 打印CT图像序列的形状
    print(f"加载后的CT图像形状: {ct_images.shape}")  # 应该是 (46, 256, 256, 3)
    img = ct_images[0]
    print(np.allclose(img[:,:,0], img[:,:,1]))  # True 表示通道值一致

    # 调整数据形状为 (batch_size, channels, depth, height, width)
    ct_images_tensor = torch.tensor(ct_images).float()  # 将图像转换为张量
    
    # permute调整数据为正确的形状 (1, 3, D, 256, 256)
    ct_images_tensor = ct_images_tensor.permute(3, 0, 1, 2).unsqueeze(0)  # (1, 3, 46, 256, 256)
    print(f"调整后的数据形状: {ct_images_tensor.shape}")  # 应该是 (1, 3, 46, 256, 256)

    # 初始化3D网络
    ct_net = CTNet3D()

    # 提取CT图像特征
    ct_features = ct_net(ct_images_tensor)

    # 打印特征维度以检查
    print(f"CT特征维度: {ct_features.shape}")  # 应该输出 (1, 256)

    save_dir = '/home/chenxi/MCAT/mydata/0101133963'

    # 设置保存路径
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 保存CT特征为.pt文件
    torch.save(ct_features, os.path.join(save_dir, 'ct_features.pt'))

    print(f"特征保存到: {save_dir}")

# 使用示例
ct_dicom_folder = '/mnt/sdc/chenxi/CT_DSA_data_new/CT/0101133963'  # 输入文件夹路径
extract_features_and_save(ct_dicom_folder)


加载后的CT图像形状: (48, 256, 256, 3)
True
调整后的数据形状: torch.Size([1, 3, 48, 256, 256])
输入数据的形状: torch.Size([1, 3, 48, 256, 256])
卷积和池化后的形状: torch.Size([1, 64, 12, 64, 64])
拉平后的形状: torch.Size([1, 3145728])
CT特征维度: torch.Size([1, 256])
特征保存到: /home/chenxi/MCAT/mydata/0101133963


In [2]:
import torch
import torch.nn as nn
import pydicom
import os
import numpy as np
from PIL import Image

# CT图像序列的预处理
def load_ct_sequence(dicom_folder, target_size=(256, 256)):
    ct_images = []
    # 读取文件夹中的所有.dcm文件
    dicom_paths = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.endswith('.dcm')]
    dicom_paths.sort()  # 按文件名排序，确保读取顺序正确
    
    for path in dicom_paths:
        dicom_img = pydicom.dcmread(path).pixel_array
        dicom_img = Image.fromarray(dicom_img).convert("RGB")
        dicom_img = dicom_img.resize(target_size)  # 调整图像大小为目标尺寸
        ct_images.append(np.array(dicom_img))  # 添加到图像列表
    
    ct_images = np.stack(ct_images, axis=0)  # 将图像堆叠成一个3D数组 (D, H, W, C)
    return ct_images

# 定义处理CT序列的3D卷积网络，输出1x256特征
class CTNet3D(nn.Module):
    def __init__(self):
        super(CTNet3D, self).__init__()
        self.conv1 = nn.Conv3d(3, 32, kernel_size=(3, 3, 3), stride=1, padding=1)  # 3D卷积
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)  # 3D池化层
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=1, padding=1)
        
        # 临时定义了一个全连接层，以后会根据输入数据的实际维度动态调整
        self.fc = None

    def forward(self, ct_images):
        print(f"输入数据的形状: {ct_images.shape}")  # 打印输入数据的形状
        x = self.conv1(ct_images)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        
        print(f"卷积和池化后的形状: {x.shape}")  # 打印卷积后的形状
        
        # 动态计算Linear层输入维度
        x = x.view(x.size(0), -1)  # 将多维数据拉平为一维
        print(f"拉平后的形状: {x.shape}")  # 打印拉平后的形状
        
        if self.fc is None:
            # 根据卷积输出的维度动态定义全连接层
            self.fc = nn.Linear(x.size(1), 256)  # 动态获取输入维度
            
        x = self.fc(x)  # 输出256维的特征
        return x

# 特征提取并保存为.pt文件
def extract_features_and_save(ct_dicom_folder, save_dir="output_features"):
    # 加载CT图像序列
    ct_images = load_ct_sequence(ct_dicom_folder)
    
    # 打印CT图像序列的形状
    print(f"加载后的CT图像形状: {ct_images.shape}")  # 应该是 (46, 256, 256, 3)

    # 调整数据形状为 (batch_size, channels, depth, height, width)
    ct_images_tensor = torch.tensor(ct_images).float()  # 将图像转换为张量
    
    # permute调整数据为正确的形状 (1, 3, D, 256, 256)
    ct_images_tensor = ct_images_tensor.permute(3, 0, 1, 2).unsqueeze(0)  # (1, 3, 46, 256, 256)
    print(f"调整后的数据形状: {ct_images_tensor.shape}")  # 应该是 (1, 3, 46, 256, 256)

    # 初始化3D网络
    ct_net = CTNet3D()

    # 提取CT图像特征
    ct_features = ct_net(ct_images_tensor)

    # 打印特征维度以检查
    print(f"CT特征维度: {ct_features.shape}")  # 应该输出 (1, 256)

    save_dir = '/home/chenxi/MCAT/mydata/0101328315'

    # 设置保存路径
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 保存CT特征为.pt文件
    torch.save(ct_features, os.path.join(save_dir, 'ct_features.pt'))

    print(f"特征保存到: {save_dir}")

# 使用示例
ct_dicom_folder = '/mnt/sdc/chenxi/CT_DSA_data_new/CT/0101328315'  # 输入文件夹路径
extract_features_and_save(ct_dicom_folder)


加载后的CT图像形状: (50, 256, 256, 3)
调整后的数据形状: torch.Size([1, 3, 50, 256, 256])
输入数据的形状: torch.Size([1, 3, 50, 256, 256])
卷积和池化后的形状: torch.Size([1, 64, 12, 64, 64])
拉平后的形状: torch.Size([1, 3145728])
CT特征维度: torch.Size([1, 256])
特征保存到: /home/chenxi/MCAT/mydata/0101328315


In [20]:
# 加载保存的特征文件
ct_features = torch.load('/home/chenxi/MCAT/mydata/CT/0000366411/ct_features.pt')

# 输出特征及其属性
print(f"加载的特征：{ct_features}")
print(f"特征的形状：{ct_features.shape}")
print(f"特征的类型：{type(ct_features)}")


加载的特征：tensor([[[ 2.6603e-01, -7.4344e-02,  3.7851e-02,  ..., -1.3511e+00,
           2.4932e-01,  2.8106e+00],
         [-9.3017e-02,  8.3702e-01, -1.9663e+00,  ...,  3.5191e+00,
           2.2294e+00,  1.5802e-01],
         [ 1.6715e+00,  3.2555e+00, -1.8328e+00,  ..., -2.1207e+00,
          -6.1300e-01, -1.3447e+00],
         ...,
         [-1.9658e-01,  4.0553e+00, -1.1452e+00,  ..., -1.8041e-01,
           3.4893e-01,  2.4084e+00],
         [ 3.1182e+00, -2.5313e-01, -1.8164e+00,  ...,  2.0466e+00,
           4.0539e-01, -4.3306e-01],
         [-2.1172e-01,  3.3293e-03, -1.0822e+00,  ...,  2.4633e+00,
           9.4007e-01, -1.3802e+00]]], requires_grad=True)
特征的形状：torch.Size([1, 10, 256])
特征的类型：<class 'torch.Tensor'>


/tmp/ipykernel_3674841/1066337397.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ct_features = torch.load('/home/chenxi/MCAT/mydata/CT/0000366411/ct_features.pt')


In [ ]:
import torch
import torch.nn as nn
import pydicom
import os
import numpy as np
from PIL import Image

# CT图像序列的预处理
def load_ct_sequence(dicom_folder, target_size=(256, 256)):
    ct_images = []
    # 读取文件夹中的所有.dcm文件
    dicom_paths = [os.path.join(dicom_folder, f) for f in os.listdir(dicom_folder) if f.endswith('.dcm')]
    dicom_paths.sort()  # 按文件名排序，确保读取顺序正确
    
    for path in dicom_paths:
        dicom_img = pydicom.dcmread(path).pixel_array
        dicom_img = Image.fromarray(dicom_img).convert("L")  # 读取为灰度图像
        dicom_img = dicom_img.resize(target_size)  # 调整图像大小为目标尺寸
        
        # 规格化图像，将像素值从 [0, 255] 缩放到 [0, 1]
        dicom_img = np.array(dicom_img) / 255.0  # 将像素值归一化到0-1之间
        
        ct_images.append(dicom_img)  # 添加到图像列表
    
    ct_images = np.stack(ct_images, axis=0)  # 将图像堆叠成一个3D数组 (D, H, W)
    return ct_images


# 定义更复杂的CT序列的3D卷积网络，输出多行特征矩阵
class ComplexCTNet3D(nn.Module):
    def __init__(self):
        super(ComplexCTNet3D, self).__init__()
        # 更复杂的卷积层结构
        self.conv1 = nn.Conv3d(1, 32, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.conv3 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.conv4 = nn.Conv3d(128, 256, kernel_size=(3, 3, 3), stride=1, padding=1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool3d(kernel_size=2, stride=2)  # 3D池化层
        
        # 动态计算全连接层的输入维度
        self.fc_final = None

    def forward(self, ct_images):
        print(f"输入数据的形状: {ct_images.shape}")  # 打印输入数据的形状
        x = self.conv1(ct_images)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv2(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv3(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.conv4(x)
        x = self.relu(x)
        x = self.pool(x)
        
        print(f"卷积和池化后的形状: {x.shape}")  # 打印卷积后的形状
        
        # 动态计算Linear层输入维度
        # 在这里，我们动态计算卷积输出后的大小，并基于此动态设置全连接层
        flat_size = np.prod(x.size()[1:])  # 获取除 batch 维度外的所有维度的乘积
        
        # 动态创建全连接层
        self.fc_final = nn.Linear(flat_size, 10 * 512)
        x = x.view(x.size(0), -1)  # 将多维数据拉平为一维
        print(f"拉平后的形状: {x.shape}")  # 打印拉平后的形状
        
        # 通过全连接层处理
        x = self.fc_final(x)  # 输出多个特征行
        
        # 将输出的特征形状变为 (batch_size, num_rows, feature_dim)
        x = x.view(x.size(0), 10, 512)  # 输出 10 行，每行 256 维特征
        return x

# 特征提取并保存为.pt文件
def extract_features_and_save(ct_dicom_folder, save_dir="output_features"):
    # 加载CT图像序列
    ct_images = load_ct_sequence(ct_dicom_folder)
    
    # 打印CT图像序列的形状
    print(f"加载后的CT图像形状: {ct_images.shape}")  # 应该是 (D, H, W)

    # 调整数据形状为 (batch_size, channels, depth, height, width)
    ct_images_tensor = torch.tensor(ct_images).float()  # 将图像转换为张量
    
    # 动态调整形状为 (1, 1, D, 256, 256)，确保batch_size=1，channels=1，depth=D，height=256，width=256
    ct_images_tensor = ct_images_tensor.unsqueeze(0).unsqueeze(0)  # (1, 1, D, 256, 256)
    print(f"调整后的数据形状: {ct_images_tensor.shape}")  # 应该是 (1, 1, D, 256, 256)

    # 初始化更复杂的3D网络
    ct_net = ComplexCTNet3D()

    # 提取CT图像特征
    ct_features = ct_net(ct_images_tensor)

    # 打印特征维度以检查
    print(f"CT特征维度: {ct_features.shape}")  # 应该输出 (1, 10, 256)

    save_dir = '/home/chenxi/MCAT/mydata/CT_2D_features_512_nol'

    # 设置保存路径
    ct_folder_name = os.path.basename(ct_dicom_folder)

    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # 保存CT特征为.pt文件
    torch.save(ct_features, os.path.join(save_dir, f'{ct_folder_name}.pt'))
    print(f"特征保存到: {save_dir}")

# 循环读取所有文件夹
def process_all_ct_folders(root_dir, save_dir="output_features"):
    # 获取所有CT文件夹
    ct_folders = [os.path.join(root_dir, f) for f in os.listdir(root_dir) if os.path.isdir(os.path.join(root_dir, f))]

    for ct_folder in ct_folders:
        print(f"正在处理: {ct_folder}")
        extract_features_and_save(ct_folder, save_dir)

# 使用示例
root_dir = '/mnt/sdc/chenxi/CT_DSA_data/CT/'  # 根目录路径
save_dir = '/home/chenxi/MCAT/mydata/CT_2D_features_512_nol'  # 保存特征的目录
process_all_ct_folders(root_dir, save_dir)


正在处理: /mnt/sdc/chenxi/CT_DSA_data/CT/0101036917
加载后的CT图像形状: (35, 256, 256)
调整后的数据形状: torch.Size([1, 1, 35, 256, 256])
输入数据的形状: torch.Size([1, 1, 35, 256, 256])
卷积和池化后的形状: torch.Size([1, 256, 2, 16, 16])
拉平后的形状: torch.Size([1, 131072])
CT特征维度: torch.Size([1, 10, 512])
特征保存到: /home/chenxi/MCAT/mydata/CT_2D_features_512/
正在处理: /mnt/sdc/chenxi/CT_DSA_data/CT/0101486244
加载后的CT图像形状: (592, 256, 256)
调整后的数据形状: torch.Size([1, 1, 592, 256, 256])
输入数据的形状: torch.Size([1, 1, 592, 256, 256])
卷积和池化后的形状: torch.Size([1, 256, 37, 16, 16])
拉平后的形状: torch.Size([1, 2424832])
CT特征维度: torch.Size([1, 10, 512])
特征保存到: /home/chenxi/MCAT/mydata/CT_2D_features_512/
正在处理: /mnt/sdc/chenxi/CT_DSA_data/CT/0101560462
加载后的CT图像形状: (385, 256, 256)
调整后的数据形状: torch.Size([1, 1, 385, 256, 256])
输入数据的形状: torch.Size([1, 1, 385, 256, 256])
卷积和池化后的形状: torch.Size([1, 256, 24, 16, 16])
拉平后的形状: torch.Size([1, 1572864])
CT特征维度: torch.Size([1, 10, 512])
特征保存到: /home/chenxi/MCAT/mydata/CT_2D_features_512/
正在处理: /mnt/sdc/chenxi/CT_DSA_d

/home/chenxi/miniconda3/envs/insertD/lib/python3.12/site-packages/torch/nn/modules/conv.py:720: UserWarning: cuDNN cannot be used for large non-batch-splittable convolutions if the V8 API is not enabled or before cuDNN version 9.3+. Consider upgrading cuDNN and/or enabling the V8 API for better efficiency. (Triggered internally at /opt/conda/conda-bld/pytorch_1729647329220/work/aten/src/ATen/native/Convolution.cpp:430.)
  return F.conv3d(


卷积和池化后的形状: torch.Size([1, 256, 73, 16, 16])
拉平后的形状: torch.Size([1, 4784128])
CT特征维度: torch.Size([1, 10, 512])
特征保存到: /home/chenxi/MCAT/mydata/CT_2D_features_512/
正在处理: /mnt/sdc/chenxi/CT_DSA_data/CT/0000366411
加载后的CT图像形状: (46, 256, 256)
调整后的数据形状: torch.Size([1, 1, 46, 256, 256])
输入数据的形状: torch.Size([1, 1, 46, 256, 256])
卷积和池化后的形状: torch.Size([1, 256, 2, 16, 16])
拉平后的形状: torch.Size([1, 131072])
CT特征维度: torch.Size([1, 10, 512])
特征保存到: /home/chenxi/MCAT/mydata/CT_2D_features_512/
正在处理: /mnt/sdc/chenxi/CT_DSA_data/CT/0101394346
加载后的CT图像形状: (689, 256, 256)
调整后的数据形状: torch.Size([1, 1, 689, 256, 256])
输入数据的形状: torch.Size([1, 1, 689, 256, 256])
卷积和池化后的形状: torch.Size([1, 256, 43, 16, 16])
拉平后的形状: torch.Size([1, 2818048])
CT特征维度: torch.Size([1, 10, 512])
特征保存到: /home/chenxi/MCAT/mydata/CT_2D_features_512/
正在处理: /mnt/sdc/chenxi/CT_DSA_data/CT/0101235241
加载后的CT图像形状: (55, 256, 256)
调整后的数据形状: torch.Size([1, 1, 55, 256, 256])
输入数据的形状: torch.Size([1, 1, 55, 256, 256])
卷积和池化后的形状: torch.Size([1, 256, 3,

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import os
from PIL import Image
import numpy as np

# 处理DSA图像，调整图像尺寸
def load_dsa_image(dsa_path, target_size=(256, 256)):
    dsa_img = Image.open(dsa_path).convert("L")  # 转为灰度图
    dsa_img = dsa_img.resize(target_size)        # 调整大小
    return np.array(dsa_img, dtype=np.float32) / 255.0  # 归一化到 [0,1]

# 定义DSA特征提取网络
class DSAFeatureExtractor(nn.Module):
    def __init__(self):
        super(DSAFeatureExtractor, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.conv1 = nn.Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        self.resnet = nn.Sequential(*list(self.resnet.children())[:-2])
        self.global_avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, 512)  # ✅ 改为 256 维输出

    def forward(self, x):
        x = self.resnet(x)              # (B, 2048, H', W')
        x = self.global_avg_pool(x)    # (B, 2048, 1, 1)
        x = x.view(x.size(0), -1)       # (B, 2048)
        x = self.fc(x)                  # (B, 256)
        x = x.unsqueeze(1)              # (B, 1, 256)
        return x

# 提取并保存单张 DSA 图像的特征
def extract_features_for_image(dsa_path, model, device, save_dir):
    img_name = os.path.splitext(os.path.basename(dsa_path))[0]
    dsa_image = load_dsa_image(dsa_path)
    dsa_tensor = torch.tensor(dsa_image).unsqueeze(0).unsqueeze(0).to(device)  # (1, 1, 256, 256)

    print(f"📂 正在处理: {dsa_path}, 输入形状: {dsa_tensor.shape}")
    model.eval()
    with torch.no_grad():
        features = model(dsa_tensor)

    print(f"✅ 特征维度: {features.shape}")
    os.makedirs(save_dir, exist_ok=True)
    save_path = os.path.join(save_dir, f"{img_name}.pt")
    torch.save(features.cpu(), save_path)
    print(f"📁 已保存特征至: {save_path}")

# 主函数：处理目录下所有 DSA 图像
def process_all_dsa_images(root_dir, save_dir="output_features", device="cuda"):
    model = DSAFeatureExtractor().to(device)
    image_files = [f for f in os.listdir(root_dir) if f.endswith(".jpg")]

    for img_file in image_files:
        img_path = os.path.join(root_dir, img_file)
        extract_features_for_image(img_path, model, device, save_dir)

# 示例调用
root_dir = '/mnt/sdc/chenxi/new_no/DSA/'  # 所有 .jpg 图像所在目录
save_dir = '/home/chenxi/MCAT/mydata/DSA_2D_features_whole'
process_all_dsa_images(root_dir, save_dir)


/home/chenxi/miniconda3/envs/insertD/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/chenxi/miniconda3/envs/insertD/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


📂 正在处理: /mnt/sdc/chenxi/new_no/DSA/0101347235.jpg, 输入形状: torch.Size([1, 1, 256, 256])
✅ 特征维度: torch.Size([1, 1, 512])
📁 已保存特征至: /home/chenxi/MCAT/mydata/DSA_2D_features_whole/0101347235.pt
📂 正在处理: /mnt/sdc/chenxi/new_no/DSA/0101481286.jpg, 输入形状: torch.Size([1, 1, 256, 256])
✅ 特征维度: torch.Size([1, 1, 512])
📁 已保存特征至: /home/chenxi/MCAT/mydata/DSA_2D_features_whole/0101481286.pt
📂 正在处理: /mnt/sdc/chenxi/new_no/DSA/0101244635.jpg, 输入形状: torch.Size([1, 1, 256, 256])
✅ 特征维度: torch.Size([1, 1, 512])
📁 已保存特征至: /home/chenxi/MCAT/mydata/DSA_2D_features_whole/0101244635.pt
📂 正在处理: /mnt/sdc/chenxi/new_no/DSA/0101405193.jpg, 输入形状: torch.Size([1, 1, 256, 256])
✅ 特征维度: torch.Size([1, 1, 512])
📁 已保存特征至: /home/chenxi/MCAT/mydata/DSA_2D_features_whole/0101405193.pt
📂 正在处理: /mnt/sdc/chenxi/new_no/DSA/0101491402.jpg, 输入形状: torch.Size([1, 1, 256, 256])
✅ 特征维度: torch.Size([1, 1, 512])
📁 已保存特征至: /home/chenxi/MCAT/mydata/DSA_2D_features_whole/0101491402.pt
📂 正在处理: /mnt/sdc/chenxi/new_no/DSA/0101550889.jpg, 输入形状

In [9]:
import torch

file_path = '/home/chenxi/MCAT/mydata/DSA_2D_features_new/0000330126.pt'
tensor_data = torch.load(file_path)
print(f"📂 文件 {file_path} 的维度: {tensor_data.shape}")


📂 文件 /home/chenxi/MCAT/mydata/DSA_2D_features_new/0000330126.pt 的维度: torch.Size([1, 1, 512])


/tmp/ipykernel_876630/2840673714.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor_data = torch.load(file_path)


In [10]:
import torch

file_path = '/home/chenxi/MCAT/mydata/CT_3D_features_new/0000283545.pt'
tensor_data = torch.load(file_path)
print(f"📂 文件 {file_path} 的维度: {tensor_data.shape}")


📂 文件 /home/chenxi/MCAT/mydata/CT_3D_features_new/0000283545.pt 的维度: torch.Size([1, 512])


/tmp/ipykernel_876630/933266233.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor_data = torch.load(file_path)
